Dynamic programming 


refeers to algorithm used to find optimal policies which have complete knowledge of the environement as an MDP


meaning : I have 


the p(s' , r | s,a)


the probability of next state and reward given the previous s and action :)


The BELLMAN EQuation 


v_pi(s)


In [4]:
# e.g. There is an agent selecting the next state from init state


init_state = 0 
states = [-2, -1, 0 , 1, 2]
action = ['left', 'right']


probability_state = { 
    'right' : {1: [0.12 , 0.22 , 0.20], 2: [0.09, 0.32, 0.05]},
    'left' : {-2: [0.34 , 0.05, 0.17], -1: [0.17, 0.23 , 0.04]}
}
# probabilioty goes like this action as key then states as key then index as reward 
# this is basically p(s', r | s, a)

probability_action = {'left': 0.4 , 'right': 0.6}

discount_rate = 0.95


states_value = {-2:19.2 , -1:16.5 , 1:18.1, 2:16.2}




In [11]:
def bellman_update(prob_dict, states_value, gamma):
    """prob_dict = probability_state['right'] or ['left']"""
    v = 0.0
    for s_prime, reward_probs in prob_dict.items():
        for r, p in enumerate(reward_probs):
            v += p * (r + gamma * states_value[s_prime])
    return v


# compute the value given each action
V_right = bellman_update(probability_state['right'], states_value, discount_rate)
V_left  = bellman_update(probability_state['left'],  states_value, discount_rate)

print("V(s | take RIGHT) =", V_right)
print("V(s | take LEFT)  =", V_left)


V(s | take RIGHT) = 17.4047
V(s | take LEFT)  = 17.8114


Bellman optimality


v*(s) = max v_pi(s)


q*(s,a) = max q_pi(s,a)

In [ ]:
# Policy Evaluation and Polcy Improvement 


list_state_val = [
    [0.0 , 0.4, 0.2, 0.1],
    [-0.2 , 0.3, -0.1, 0.8],
    [0.9, -0.2, 0.6, 0.1],
    [0.1, 0.9, -0.9 , 0.0]
]

Rt = -1
pi_a_s = 0.25 # four direction equal probability


# it doesnt satisfy bellman equation not valid state values
action = [
    [-1, 0],
    [1, 0],
    [0, 1],
    [0, -1],
] 

for i in range(len(list_state_val)):
    for j in range(len(list_state_val[i])):
        if list_state_val[i][j] == 0:
            continue
        print(list_state_val[i][j])
        sum = 0 
        for act in action:
            i, j = i-act[0] , j -act[1]
            # clip them 

            sum += 

0.4
0.2
0.1
-0.2
0.3
-0.1
0.8
0.9
-0.2
0.6
0.1
0.1
0.9
-0.9


In [16]:
import numpy as np

# given V (4x4)
list_state_val = [
    [0.0 , 0.4, 0.2, 0.1],
    [-0.2 , 0.3, -0.1, 0.8],
    [0.9, -0.2, 0.6, 0.1],
    [0.1, 0.9, -0.9 , 0.0]
]
V = np.array(list_state_val, dtype=float)      # shape (4,4)

rows, cols = V.shape
Rt = -1
pi_a_s = 0.25  # uniform over 4 actions
gamma = 0.9    # discount factor

# actions: up, down, right, left
actions = [(-1,0),(1,0),(0,1),(0,-1)]

# terminal states (top-left and bottom-right)
terminal_states = {(0,0), (rows-1, cols-1)}

# ensure terminal V is zero
for (ti, tj) in terminal_states:
    V[ti, tj] = 0.0

def clip(i,j):
    return min(max(0,i), rows-1), min(max(0,j), cols-1)

# Build deterministic p(s',r|s,a)
# For terminal states: treat as absorbing with reward 0 (self-loop)
p = {}
for i in range(rows):
    for j in range(cols):
        p[(i,j)] = {}
        for a_idx, (di,dj) in enumerate(actions):
            if (i,j) in terminal_states:
                # absorbing: stays in terminal and gives 0 reward
                p[(i,j)][a_idx] = { (i,j): { 0: 1.0 } }
            else:
                ni, nj = clip(i+di, j+dj)
                # deterministic next-state with reward Rt (non-terminal)
                p[(i,j)][a_idx] = { (ni, nj): { Rt: 1.0 } }

# Compute Bellman RHS for each state under the uniform policy
RHS = np.zeros_like(V, dtype=float)
for i in range(rows):
    for j in range(cols):
        # If terminal, we keep RHS = V (or compute via self-loop, same effect)
        total = 0.0
        for a_idx in range(len(actions)):
            trans = p[(i,j)][a_idx]
            for (ni, nj), reward_dict in trans.items():
                for r, prob in reward_dict.items():
                    total += pi_a_s * prob * (r + gamma * V[ni, nj])
        RHS[i, j] = total

# Residual = RHS - V : should be 0 for states that already satisfy Bellman
residual = RHS - V

np.set_printoptions(precision=4, suppress=True)
print("V (given):\n", V)
print("\nBellman RHS under deterministic model:\n", RHS)
print("\nResidual = RHS - V:\n", residual)


V (given):
 [[ 0.   0.4  0.2  0.1]
 [-0.2  0.3 -0.1  0.8]
 [ 0.9 -0.2  0.6  0.1]
 [ 0.1  0.9 -0.9  0. ]]

Bellman RHS under deterministic model:
 [[ 0.     -0.7975 -0.865  -0.73  ]
 [-0.775  -1.0225 -0.5725 -0.7975]
 [-0.865  -0.3925 -1.2475 -0.6625]
 [-0.55   -1.0225 -0.865   0.    ]]

Residual = RHS - V:
 [[ 0.     -1.1975 -1.065  -0.83  ]
 [-0.575  -1.3225 -0.4725 -1.5975]
 [-1.765  -0.1925 -1.8475 -0.7625]
 [-0.65   -1.9225  0.035   0.    ]]
